# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Perotti  ->  G. Perotti  |  ['G. Perotti']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
J. Li  ->  J. Li  |  ['J. Li']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
K. Paschek  ->  K. Paschek  |  ['K. Paschek']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
Arxiv has 62 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.11074


extracting tarball to tmp_2310.11074...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.11074/XUE1_referee.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgments' from 'tmp_2310.11074/acknowledgments.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors' from 'tmp_2310.11074/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packa

...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.11433


extracting tarball to tmp_2310.11433...

 done.


Found 138 bibliographic references in tmp_2310.11433/main.bbl.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Chyba1992}C.~Chyba, C.~Sagan, \emph{Nature} \textbf{1992}, \emph{355}, 125.
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem{Morbidelli2015}A.~Morbidelli, B.~J. Wood, Late {{Accretion}} and the {{Late Veneer}}, in \emph{The {{Early Earth}}: Accretion and Differentiation}, chapter~4, pages 71--82, {American Geophysical Union (AGU)} \textbf{2015}.
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/si

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.11074-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.11074) | **XUE. Molecular inventory in the inner region of an extremely irradiated  Protoplanetary Disk**  |
|| M. C. Ramirez-Tannus, et al. -- incl., <mark>G. Perotti</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2023-10-18*|
|*Comments*| *Accepted for publication in ApJ Letters. 20 pages, 7 figures*|
|**Abstract**| We present the first results of the eXtreme UV Environments (XUE) James Webb Space Telescope (JWST) program, that focuses on the characterization of planet forming disks in massive star forming regions. These regions are likely representative of the environment in which most planetary systems formed. Understanding the impact of environment on planet formation is critical in order to gain insights into the diversity of the observed exoplanet populations. XUE targets 15 disks in three areas of NGC 6357, which hosts numerous massive OB stars, among which some of the most massive stars in our Galaxy. Thanks to JWST we can, for the first time, study the effect of external irradiation on the inner ($< 10$ au), terrestrial-planet forming regions of proto-planetary disks. In this study, we report on the detection of abundant water, CO, CO$_2$, HCN and C$_2$H$_2$ in the inner few au of XUE 1, a highly irradiated disk in NGC 6357. In addition, small, partially crystalline silicate dust is present at the disk surface. The derived column densities, the oxygen-dominated gas-phase chemistry, and the presence of silicate dust are surprisingly similar to those found in inner disks located in nearby, relatively isolated low-mass star-forming regions. Our findings imply that the inner regions of highly irradiated disks can retain similar physical and chemical conditions as disks in low-mass star-forming regions, thus broadening the range of environments with similar conditions for inner disk rocky planet formation to the most extreme star-forming regions in our Galaxy. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.11433-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.11433) | **Prebiotic Vitamin B$_3$ Synthesis in Carbonaceous Planetesimals**  |
|| <mark>K. Paschek</mark>, M. Lee, D. A. Semenov, <mark>T. K. Henning</mark> |
|*Appeared on*| *2023-10-18*|
|*Comments*| *Accepted for publication in ChemPlusChem. The authors Klaus Paschek and Mijin Lee contributed equally. 18 pages, 7 figures (all colored). Supporting Information is available at this https URL*|
|**Abstract**| Aqueous chemistry within carbonaceous planetesimals is promising for synthesizing prebiotic organic matter essential to all life. Meteorites derived from these planetesimals delivered these life building blocks to the early Earth, potentially facilitating the origins of life. Here, we studied the formation of vitamin B$_3$ as it is an important precursor of the coenzyme NAD(P)(H), which is essential for the metabolism of all life as we know it. We propose a new reaction mechanism based on known experiments in the literature that explains the synthesis of vitamin B$_3$. It combines the sugar precursors glyceraldehyde or dihydroxyacetone with the amino acids aspartic acid or asparagine in aqueous solution without oxygen or other oxidizing agents. We performed thermochemical equilibrium calculations to test the thermodynamic favorability. The predicted vitamin B$_3$ abundances resulting from this new pathway were compared with measured values in asteroids and meteorites. We conclude that competition for reactants and decomposition by hydrolysis are necessary to explain the prebiotic content of meteorites. In sum, our model fits well into the complex network of chemical pathways active in this environment. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.11412-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.11412) | **Stellar mass-metallicity relation throughout the large-scale of the  Universe: CAVITY mother sample**  |
|| J. Domínguez-Gómez, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2023-10-18*|
|*Comments*| *Accepted in A&A, 39 pages, 18 figures*|
|**Abstract**| Void galaxies are essential to understand the physical processes that drive galaxy evolution as they are less affected by external factors than galaxies in denser environments, i.e. filaments, walls, and clusters. The stellar metallicity of a galaxy traces the accumulated fossil record of star formation through its entire life. Comparing the stellar metallicity of galaxies in various environments, including voids, filaments, walls, and clusters, can provide valuable insights into how the large-scale environment impacts galaxy chemical evolution. We present the first comparison of the total stellar mass vs. central stellar metallicity relation between galaxies in voids, filaments, walls, and clusters with different star formation history (SFH) types, morphologies, and colours, for stellar masses between 10^8.0 to 10^11.5 solar masses and redshift 0.01 < z < 0.05. We aim to better understand how the large-scale structure affects galaxy evolution by studying the stellar mass-metallicity relation of thousands of galaxies, which allows us to make a statistically sound comparison between galaxies in voids, filaments, walls, and clusters. We apply non-parametric full spectral fitting techniques (pPXF and STECKMAP) to 10807 spectra from the SDSS-DR7 (987 in voids, 6463 in filaments and walls, and 3357 in clusters) and derive their central mass-weighted average stellar metallicity. We find that galaxies in voids have on average slightly lower stellar metallicities than galaxies in filaments and walls (by 0.1 dex), and much lower than galaxies in clusters (by 0.4 dex). These differences are more significant for low-mass (10^9.25) than for high-mass galaxies, for long-timescale SFH (LT-SFH, extended along time) galaxies than for short-timescale SFHs (ST-SFH, concentrated at early times) galaxies, for spiral than for elliptical galaxies, and for blue than for red galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.11391-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.11391) | **Constraints on Axion-like Particles from the Observation of GRB 221009A  by LHAASO**  |
|| L.-Q. Gao, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-10-18*|
|*Comments*| *8 pages, 13 figures*|
|**Abstract**| Recently, the LHAASO collaboration reported the measurement of the gamma-ray spectra of GRB 221009A, which is the brightest burst ever, covering an energy range from 0.3 to about 10 TeV. Based on the observation by LHAASO, we examine the ALP-photon oscillation effect in the host galaxy of GRB 221009A and Milky Way. Considering the uncertain magnetic field configuration in the host galaxy, we set constraints on the ALP parameters with three different models: a homogeneous magnetic field model, a model same as the Galactic magnetic field model, and a model constructed from the HST observations of the host halo. The ${\rm CL_s}$ method is applied to set the constraints in this study. We find that the constraints derived with three host galaxy magnetic field models are comparable. Our results are complementary in the small ALP mass regions compared with other experiments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.11074.md
    + _build/html/tmp_2310.11074/./Pis24_extinction_Av8p7.png
    + _build/html/tmp_2310.11074/./F850W_F560Wimage.png
    + _build/html/tmp_2310.11074/./MIRI_spectrum.png
exported in  _build/html/2310.11433.md
    + _build/html/tmp_2310.11433/./structures.png
    + _build/html/tmp_2310.11433/./aldehyde_variable_3-Oxopropanoic_acid_100bar_amounts.png
    + _build/html/tmp_2310.11433/./plot_additional_Gibbs.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\nodata}{...}$
$\newcommand{\Hii}{H {\sc ii}}$
$\newcommand{\micron}{\mum}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\cmss}{cm s^{-2}}$
$\newcommand{\lsol}{L_{\odot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msol}{M_{\odot}}$
$\newcommand{\rsol}{R_{\odot}}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\s}{\sigma}$
$\newcommand{\w}{\omega}$
$\newcommand{\vsini}{v \sin i}$
$\newcommand{\sigrms}{\sigma_\mathrm{rms}}$
$\newcommand{\srv}{\sigma_\mathrm{RV}}$
$\newcommand{\Msol}{M_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Lsol}{L_\odot}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\s}{\sigma}$
$\newcommand{\feros}{{\sc feros}}$
$\newcommand{\lco}{{\sc lco}}$
$\newcommand{\uves}{{\sc uves}}$
$\newcommand{\iacob}{{\sc iacob}}$
$\newcommand{\l}{\lambda}$
$\newcommand{\ll}{\lambda\lambda}$
$\newcommand{\palp}{Pa~\alpha}$
$\newcommand{\palph}{Pa~\alpha}$
$\newcommand{\palpha}{Pa~\alpha}$
$\newcommand{\pbet}{Pa~\beta}$
$\newcommand{\pbeta}{Pa~\beta}$
$\newcommand{\pdelt}{Pa~\delta}$
$\newcommand{\pgam}{Pa~\gamma}$
$\newcommand{\peps}{Pa~\epsilon}$
$\newcommand{\halp}{H~\alpha}$
$\newcommand{\halph}{H~\alpha}$
$\newcommand{\halpha}{H~\alpha}$
$\newcommand{\hbet}{H~\beta}$
$\newcommand{\hdelt}{H~\delta}$
$\newcommand{\hgam}{H~\gamma}$
$\newcommand{\ha}{H {\sc i}}$
$\newcommand{\hb}{H {\sc ii}}$
$\newcommand{\hea}{He {\sc i}}$
$\newcommand{\heb}{He {\sc ii}}$
$\newcommand{\nc}{N {\sc iii}}$
$\newcommand{\fea}{Fe {\sc i}}$
$\newcommand{\nd}{N {\sc iv}}$
$\newcommand{\ne}{N {\sc v}}$
$\newcommand{\mgb}{Mg {\sc ii}}$
$\newcommand{\ob}{O {\sc ii}}$
$\newcommand{\sic}{Si {\sc iii}}$
$\newcommand{\sid}{Si {\sc iv}}$
$\newcommand{\H2O}{H_{2}O}$
$\newcommand{\C2H2}{C_{2}H_{2}}$
$\newcommand{\CO2}{^{12}CO_{2}}$
$\newcommand{\13CO}{^{13}CO_{2}}$
$\newcommand{\13CO2}{^{13}CO_{2}}$</div>



<div id="title">

# XUE. Molecular inventory in the inner region of an extremely irradiated Protoplanetary Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.11074-b31b1b.svg)](https://arxiv.org/abs/2310.11074)<mark>Appeared on: 2023-10-18</mark> -  _Accepted for publication in ApJ Letters. 20 pages, 7 figures_

</div>
<div id="authors">

M. C. Ramírez-Tannus, et al. -- incl., <mark>G. Perotti</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** We present the first results of the eXtreme UV Environments (XUE) James Webb Space Telescope (JWST) program, that focuses on the characterization of planet forming disks in massive star forming regions. These regions are likely representative of the environment in which most planetary systems formed. Understanding the impact of environment on planet formation is critical in order to gain insights into the diversity of the observed exoplanet populations.  XUE targets 15 disks in three areas of NGC 6357, which hosts numerous massive OB stars, among which some of the most massive stars in our galaxy. Thanks to JWST we can, for the first time, study the effect of external irradiation on the inner ( $< 10$ au), terrestrial-planet forming regions of proto-planetary disks. In this study, we report on the detection of abundant water, CO, $\CO$ 2, HCN and $\C$ 2H2 in the inner few au of XUE 1, a highly irradiated disk in NGC 6357. In addition, small, partially crystalline silicate dust is present at the disk surface.The derived column densities, the oxygen-dominated gas-phase chemistry, and the presence of silicate dust are surprisingly similar to those found in inner disks located in nearby, relatively isolated low-mass star-forming regions. Our findings imply that the inner regions of highly irradiated disks can retain similar physical and chemical conditions as disks in low-mass star-forming regions, thus broadening the range of environments with similar conditions for inner disk rocky planet formation to the most extreme star-forming regions in our Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2310.11074/./Pis24_extinction_Av8p7.png" alt="Fig2" width="100%"/>

**Figure 2. -** _Top:_ Extinction $A_K$ for a sample of stars in Pis24, $A_K$ is shown in colors. The position of XUE 1 in this diagram is indicated with a star. The O stars from the bottom panel are indicated with magenta borders. _Bottom:_ Radiation field towards XUE 1. The lines show the 2D distance from the massive stars to XUE 1 (indicated with the black star) and the colors of the dots show their temperature. The FUV radiation felt by XUE 1 from each massive star is shown by the color and width =of the lines (*fig:ext_radiation*)

</div>
<div id="div_fig2">

<img src="tmp_2310.11074/./F850W_F560Wimage.png" alt="Fig3" width="100%"/>

**Figure 3. -** _Left:_ HST/ACS  F850LP band image of the target position for XUE 1.
   The three point-like objects are marked by green circles
with radii of $0.1$\arcsec$$.
The cyan circle on A1  marks the position of the Gaia DR3 source 5976051168205228416.
The magenta circle ($0.5$\arcsec$$ radius) marks the position of the
_Chandra_ X-ray source. A grid of J2000 coordinates is shown.
_Right:_ MIRI F560W image (log intensity scale) of the target position for XUE 1. The white box marks the observed field of view with MRS. The optical positions of the three point-like objects A1, A2, and B are marked by green circles
with radii of $0.1$\arcsec$$. (*fig:HST-MIRI-images*)

</div>
<div id="div_fig3">

<img src="tmp_2310.11074/./MIRI_spectrum.png" alt="Fig4" width="100%"/>

**Figure 4. -** MIRI MRS spectrum of XUE 1. The most prominent dust features are labeled. The insets show the P-branch transitions of the CO ro-vibrational fundamental band, the water emission around 7 $\micron$ and the 13 to 15 $\micron$ region featuring $\C$2H2, HCN, and $\C$O2.  (*fig:MIRIspectrum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.11074"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\affiliation}{$
$\begin{itemize}$
$\item[{[a]}] K. Paschek*, M. Lee, Dr. D. A. Semenov, Prof. T. K. Henning\Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany\E-mail: paschek@mpia.de\item[{[b]}] Dr. D. A. Semenov\Department of Chemistry, Ludwig Maximilian University of Munich, Butenandtstraße 5-13, House F, D-81377 Munich, Germany\item[{[\texttt{+}]}] These authors contributed equally.$
$\end{itemize}$
$}$
$\newcommand{\keywords}{$
$    Meteorites \textbullet     Nitrogen heterocycles \textbullet     Origins of life \textbullet     Prebiotic chemistry \textbullet     Thermochemistry$
$}$
$\newcommand{\dedication}{$
$	\begin{minipage}{\textwidth}$
$	\end{minipage}$
$}$
$\newcommand{\abstract}{Aqueous chemistry within carbonaceous planetesimals is promising for synthesizing prebiotic organic matter essential to all life. Meteorites derived from these planetesimals delivered these life building blocks to the early Earth, potentially facilitating the origins of life. Here, we studied the formation of vitamin B_3 as it is an important precursor of the coenzyme NAD(P)(H), which is essential for the metabolism of all life as we know it. We propose a new reaction mechanism based on known experiments in the literature that explains the synthesis of vitamin B_3. It combines the sugar precursors glyceraldehyde or dihydroxyacetone with the amino acids aspartic acid or asparagine in aqueous solution without oxygen or other oxidizing agents. We performed thermochemical equilibrium calculations to test the thermodynamic favorability. The predicted vitamin B_3 abundances resulting from this new pathway were compared with measured values in asteroids and meteorites. We conclude that competition for reactants and decomposition by hydrolysis are necessary to explain the prebiotic content of meteorites. In sum, our model fits well into the complex network of chemical pathways active in this environment.}$</div>



<div id="title">

# $\raggedright$ Prebiotic Vitamin B$_\text{3}$ Synthesis in Carbonaceous Planetesimals

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.11433-b31b1b.svg)](https://arxiv.org/abs/2310.11433)<mark>Appeared on: 2023-10-18</mark> -  _Accepted for publication in ChemPlusChem. The authors Klaus Paschek and Mijin Lee contributed equally. 18 pages, 7 figures (all colored). Supporting Information is available at this https URL_

</div>
<div id="authors">

<mark>K. Paschek</mark>, M. Lee, D. A. Semenov, <mark>T. K. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Aqueous chemistry within carbonaceous planetesimals is promising for synthesizing prebiotic organic matter essential to all life. Meteorites derived from these planetesimals delivered these life building blocks to the early Earth, potentially facilitating the origins of life. Here, we studied the formation of vitamin $B_3$ as it is an important precursor of the coenzyme NAD(P)(H), which is essential for the metabolism of all life as we know it. We propose a new reaction mechanism based on known experiments in the literature that explains the synthesis of vitamin $B_3$. It combines the sugar precursors glyceraldehyde or dihydroxyacetone with the amino acids aspartic acid or asparagine in aqueous solution without oxygen or other oxidizing agents. We performed thermochemical equilibrium calculations to test the thermodynamic favorability. The predicted vitamin $B_3$ abundances resulting from this new pathway were compared with measured values in asteroids and meteorites. We conclude that competition for reactants and decomposition by hydrolysis are necessary to explain the prebiotic content of meteorites. In sum, our model fits well into the complex network of chemical pathways active in this environment. 

</div>

<div id="div_fig1">

<img src="tmp_2310.11433/./structures.png" alt="Fig2" width="100%"/>

**Figure 2. -** Structures of **A** vitamin $B_3$(in box) and its isomers, and **B** the coenzyme nicotinamide adenine dinucleotide (phosphate), abbreviated as NAD(P)H, in its reduced form. The phosphorylated form is indicated in red. (*fig:structures*)

</div>
<div id="div_fig2">

<img src="tmp_2310.11433/./aldehyde_variable_3-Oxopropanoic_acid_100bar_amounts.png" alt="Fig5" width="100%"/>

**Figure 5. -** Simulated vitamin $B_3$ abundances depending on variable initial concentrations of the aldehydes 3-oxopropanoic acid (\ce{C3H4O3}) and 3-oxopropanamide (\ce{C3H5NO2}), which are reactants in the Strecker synthesis (**S1** in \cref{sch:reaction}). All simulations were performed at \SI{0}{\celsius} and \SI{100}{bar}. The vertical dotted black line indicates the initial concentration of propanal (mean of the range given in \cref{tab:concs}), which was used in the simulations as a surrogate for these aldehydes that have not yet been detected in comets. (*fig:variable_aldehyde*)

</div>
<div id="div_fig3">

<img src="tmp_2310.11433/./plot_additional_Gibbs.png" alt="Fig6" width="100%"/>

**Figure 6. -** Gibbs free energies of formation $\Delta G_{f,\mathrm{(aq)}}$ as a function of temperature $T$ for the molecules not included in the CHNOSZ database. All energies are given in an aqueous solution at a pressure of \SI{100}{bar}, assuming an ideal infinite dilution. (*fig:Gibbs_add*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.11433"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

302  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2

  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
